In [323]:
#Imports
import time
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
import urllib.request
from urllib.request import urlopen
import pyautogui
from requests.exceptions import ConnectionError
# from pywebcopy import save_webpage

In [324]:

internal_urls = set()
external_urls = set()

In [325]:
def is_url_valid(url):
   #Checks if the url is valid or not
    parsedURL = urlparse(url)
    return bool(parsedURL.netloc) and bool(parsedURL.scheme)

#The above function works as follows. We check if the scheme is present and there is a value in the network location part
# url = "https://umd.edu/virusinfo"
# urlparse(url)
# ParseResult(scheme='https', netloc='umd.edu', path='/virusinfo', 
# params='', query='', fragment='')

In [326]:

#The function gives all urls
def get_all_urls(url):
    urls = set()
    # domain name of the URL without the protocol (umd.edu in this case)
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            #href is empty and we don't need that a element
            continue
        #if the link is not absolute, make it by joining relative to the base
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        #constructing an absolute URL from parsed data
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_url_valid(href):
            #in valid url
            continue
        if href in internal_urls:
            #it is already in the set, so we don't need to add
            continue
        if domain_name not in href:
            #it is an external link. i.e
            # Check if it is already there 
            if href not in external_urls:
#                 print(f"[EXT] External link: {href}")
                external_urls.add(href)
            continue
#         print(".",end=" ")
        print(f"[INT] Internal link: {href}")

        urls.add(href)
        internal_urls.add(href)
    return urls


In [327]:
total_urls_visited = 0
def crawl(url, max_urls=50):
    #Max URL is just to decrease the time if there are a lot of pages.
    #The following code was openly available of github and I found this
    #idea useful to inhibit crawling time
    global total_urls_visited
    total_urls_visited += 1
    try:
        links = get_all_urls(url)
        for link in links:
            if total_urls_visited > max_urls:
                break
            crawl(link, max_urls=max_urls)
    except:
        print("caught")


In [328]:
#base_url = "https://umd.edu/virusinfo"
#Input
# base_url = input("Enter the URL : ")
base_url = "https://covid19.yale.edu/"
internal_urls.add(base_url)

In [329]:
parsedurl = urlparse(base_url)
print(parsedurl)
base_url_main = parsedurl.scheme+"://"+parsedurl.netloc
print(base_url_main)
# base_url_text = base_url.split("//",1)[1]
base_url_text = parsedurl.netloc+parsedurl.path
print(base_url_text)
# base_url_text_domain = base_url_text.split("/",1)[0]
base_url_text_domain = parsedurl.netloc
print(base_url_text_domain)

ParseResult(scheme='https', netloc='covid19.yale.edu', path='/', params='', query='', fragment='')
https://covid19.yale.edu
covid19.yale.edu/
covid19.yale.edu


In [330]:
# internal_urls

In [ ]:
crawl(base_url)
print("[+] Total External links:", len(external_urls))
print("[+] Total Internal links:", len(internal_urls))
print("[+] Total:", len(external_urls) + len(internal_urls))

[INT] Internal link: https://covid19.yale.edu/university-announcements
[INT] Internal link: https://covid19.yale.edu/health-safety-guidelines
[INT] Internal link: https://covid19.yale.edu/campus-resources
[INT] Internal link: https://covid19.yale.edu/yale-and-new-haven
[INT] Internal link: https://covid19.yale.edu/contact
[INT] Internal link: https://covid19.yale.edu/announcements/fall-2020
[INT] Internal link: https://covid19.yale.edu/announcements/covid-19-contingency-planning-committees
[INT] Internal link: https://covid19.yale.edu/screening
[INT] Internal link: https://covid19.yale.edu/announcements/announcements-other-university-leaders-related-fall-2020
[INT] Internal link: https://covid19.yale.edu/faqs
[INT] Internal link: https://covid19.yale.edu/sites/default/files/files/harold-koh-2.jpg
[INT] Internal link: https://covid19.yale.edu/covid-19-signs-posters-campus
[INT] Internal link: https://covid19.yale.edu/yales-covid-19-contact-tracing-program
[INT] Internal link: https://co

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [ ]:
count = 0
last_count = 989874
immediate_urls = set() #Linked Associated with the current page, https://umd.edu/virusinfo/ in this case

for i in range(0,4):
    for url in internal_urls:
        if base_url_text in url:
            immediate_urls.add(url)
    count = len(immediate_urls)
    print(count,i)
    if(count == last_count):
        break
    last_count = count
    for immediate_url in immediate_urls:
        try:
            crawl(immediate_url)
            print("[+] NEW Total External links:", len(external_urls))
            print("[+] NEW Total Internal links:", len(internal_urls))
            print("[+] NEW Total:", len(external_urls) + len(internal_urls))
            if len(internal_urls)>1000:
                break
        except requests.exceptions.ConnectionError:
            continue

In [ ]:
def removeAfterN(yourStr, nth, occurenceOf):
    return occurenceOf.join(yourStr.split(occurenceOf)[:nth])

In [ ]:
semifinal_immediate_urls = set()

for immediate_url in immediate_urls:
    stripped_url = removeAfterN(immediate_url,2,"http")
    
    
    semifinal_immediate_urls.add(stripped_url)
#     final_immediate_urls.add(rest)
#     print(rest)


    
for i in semifinal_immediate_urls:
    print(i)
total_count = len(semifinal_immediate_urls)
print(total_count)

In [ ]:
final_immediate_urls = set()
for semifinal_immediate_url in semifinal_immediate_urls:
    if "tel:" not in semifinal_immediate_url:
        final_immediate_urls.add(semifinal_immediate_url)


for i in final_immediate_urls:
    print(i)
total_count = len(final_immediate_urls)
print(total_count)       

In [ ]:

parent_folder = "Doctoral" #Change based on folder
name_of_folder = f'{base_url_text_domain}_Screenshots'
options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options,executable_path='chromedriver.exe') # #Local Path of chrome driver
if not os.path.exists(f'{parent_folder}\\{name_of_folder}'):
    os.makedirs(f'{parent_folder}\\{name_of_folder}')

In [ ]:
webpage_save = set()
webpage_save_count = 0
webpage_cannot_save = 0
for i in final_immediate_urls:
    if ' ' in i:
        continue
    try:
        code=urlopen(i).getcode()
        webpage_save.add(i)
        webpage_save_count = webpage_save_count+1
        print(f"{code} can_save - {i}")
    except (urllib.error.HTTPError,urllib.error.URLError) as e:
        print("discarded")
        webpage_cannot_save = webpage_cannot_save+1
        continue

print(f'Webpages save as HTML :{webpage_save_count}')
print(f'Webpages unable to save as HTML :{webpage_cannot_save}')
        

In [ ]:
pdf_count = 0
pdf_urls = set()
for webpage in webpage_save:
    try:
        if webpage[-4:] == ".pdf":
            url = webpage
            file_name = url.replace(base_url,'')
            file_name = file_name.replace('/','')
            if file_name == '':
                file_name = "Home"
            pdf_urls.add(webpage)
            print(webpage)
            r = requests.get(url, stream=True)
            with open(f'{parent_folder}\\{name_of_folder}\\{file_name}.pdf', 'wb') as fd:
                for chunk in r.iter_content(2000):
                    fd.write(chunk)
            pdf_count = pdf_count + 1
    except:
        print("caught")

print(f"Pdfs saved: {pdf_count}")

In [ ]:
final_immediate_urls = final_immediate_urls - pdf_urls
webpage_save = webpage_save - pdf_urls
web_save_count = len(webpage_save)
total_count = len(final_immediate_urls)
print(total_count) 

In [ ]:

#     try:
#         print(i)
        
#         print("added")
#     except urllib.error.HTTPError as e:
#         if e.code in (400,403):
#             print("discarded")
#             continue

In [ ]:
final_webpage_save_count = webpage_save_count - pdf_count
print(final_webpage_save_count)

In [ ]:
webpage_save_counter_loop = 0
for webpage in webpage_save:
    try:
        url = webpage
        file_name = url.replace(base_url,'')
        file_name = file_name.replace('/','')
        if file_name == '':
                file_name = "Home"
        urllib.request.urlretrieve (webpage, f"{parent_folder}\\{name_of_folder}\\{file_name}.html")
        webpage_save_counter_loop = webpage_save_counter_loop+1
        print(f'{webpage_save_counter_loop} of {final_webpage_save_count} - {webpage}')
    except:
        print("caught")

print("Saving Completed")

In [322]:
count = 0
for i in final_immediate_urls:
    try:
        url = i
        file_name = url.replace(base_url,'')
        file_name = file_name.replace('/','')
        if file_name == '':
                file_name = "Home"
        count = count + 1
        print(f"{count} of {total_count}")
        print(f'Visiting {base_url_text}/{file_name}')
        print(f'...Taking a screenshot')
        driver.get(url)
        S = lambda X: driver.execute_script('return document.body.parentNode.scroll'+X)
        driver.set_window_size(S('Width'),S('Height')) # May need manual adjustment                                                                                                                
        driver.find_element_by_tag_name('body').screenshot(f'{parent_folder}\\{name_of_folder}\\{file_name}.png')
        print(f'Screenshot of {file_name} page taken! \n')
    except:
        print("caught")
#     print("Saving HTML")
#     pyautogui.hotkey('ctrl', 's')
#     time.sleep(1)
#     pyautogui.typewrite(f'{file_name}.html')
#     pyautogui.hotkey('enter')
driver.quit()
    
print(f'Task Completed! Files stored in the {parent_folder}\\{name_of_folder} Folder')

1 of 172
Visiting www.vanderbilt.edu/coronavirus//20200307mar-7-2020-vice-provost-for-faculty-affairs-message-about-continuity-planning-for-teaching
...Taking a screenshot
Screenshot of 20200307mar-7-2020-vice-provost-for-faculty-affairs-message-about-continuity-planning-for-teaching page taken! 

2 of 172
Visiting www.vanderbilt.edu/coronavirus//20200514may-14-2020-take-action-returning-to-retrieve-your-belongings
...Taking a screenshot
Screenshot of 20200514may-14-2020-take-action-returning-to-retrieve-your-belongings page taken! 

3 of 172
Visiting www.vanderbilt.edu/coronavirus//20200311resources-for-students-others-on-campus-through-march-30
...Taking a screenshot
Screenshot of 20200311resources-for-students-others-on-campus-through-march-30 page taken! 

4 of 172
Visiting www.vanderbilt.edu/coronavirus//20200501may-1-2020-information-forthcoming-on-returning-for-belongings
...Taking a screenshot
Screenshot of 20200501may-1-2020-information-forthcoming-on-returning-for-belongings 

Screenshot of faqslabs page taken! 

41 of 172
Visiting www.vanderbilt.edu/coronavirus//fall-2020-resourcesresources-for-vu-communicators
...Taking a screenshot
Screenshot of fall-2020-resourcesresources-for-vu-communicators page taken! 

42 of 172
Visiting www.vanderbilt.edu/coronavirus//20200325vanderbilt-community-update-on-commencement
...Taking a screenshot
Screenshot of 20200325vanderbilt-community-update-on-commencement page taken! 

43 of 172
Visiting www.vanderbilt.edu/coronavirus//20200605june-5-2020-transition-to-vanderbilt-phase-2-of-return-to-campus-plan-begins-june-8
...Taking a screenshot
Screenshot of 20200605june-5-2020-transition-to-vanderbilt-phase-2-of-return-to-campus-plan-begins-june-8 page taken! 

44 of 172
Visiting www.vanderbilt.edu/coronavirus//fall-2020fall-2020-faculty
...Taking a screenshot
Screenshot of fall-2020fall-2020-faculty page taken! 

45 of 172
Visiting www.vanderbilt.edu/coronavirus//faqscommunity-support
...Taking a screenshot
Screenshot of faq

Screenshot of faqsvanderbilt-return-to-campus-phase-two-continue page taken! 

83 of 172
Visiting www.vanderbilt.edu/coronavirus//20200605june-5-2020-transition-to-phase-2-of-research-ramp-up-plan-begins-next-week
...Taking a screenshot
Screenshot of 20200605june-5-2020-transition-to-phase-2-of-research-ramp-up-plan-begins-next-week page taken! 

84 of 172
Visiting www.vanderbilt.edu/coronavirus//travel-returning
...Taking a screenshot
Screenshot of travel-returning page taken! 

85 of 172
Visiting www.vanderbilt.edu/coronavirus//fall-2020-faculty
...Taking a screenshot
Screenshot of fall-2020-faculty page taken! 

86 of 172
Visiting www.vanderbilt.edu/coronavirus//20200312mar-12-2020-further-information-about-move-out
...Taking a screenshot
Screenshot of 20200312mar-12-2020-further-information-about-move-out page taken! 

87 of 172
Visiting www.vanderbilt.edu/coronavirus//faqsschool-and-childcare-working-group
...Taking a screenshot
Screenshot of faqsschool-and-childcare-working-group

Screenshot of 20200309mar-9-2020-classes-suspended page taken! 

122 of 172
Visiting www.vanderbilt.edu/coronavirus//fall-2020fall-2020-resourcesfall-2020-graduate-and-professional-academic-calendars
...Taking a screenshot
Screenshot of fall-2020fall-2020-resourcesfall-2020-graduate-and-professional-academic-calendars page taken! 

123 of 172
Visiting www.vanderbilt.edu/coronavirus//20200515may-15-2020-fall-2020-study-abroad-suspended
...Taking a screenshot
Screenshot of 20200515may-15-2020-fall-2020-study-abroad-suspended page taken! 

124 of 172
Visiting www.vanderbilt.edu/coronavirus//20200320mar-20-2020-room-and-board-adjustments-student-hardship-fund-launching
...Taking a screenshot
Screenshot of 20200320mar-20-2020-room-and-board-adjustments-student-hardship-fund-launching page taken! 

125 of 172
Visiting www.vanderbilt.edu/coronavirus//20200213feb-13-2020-information-for-postdocs-from-china
...Taking a screenshot
Screenshot of 20200213feb-13-2020-information-for-postdocs-from-c

Screenshot of fall-2020fall-2020-staff page taken! 

162 of 172
Visiting www.vanderbilt.edu/coronavirus//faqsacademic-calendar-for-graduate-and-professional-students
...Taking a screenshot
Screenshot of faqsacademic-calendar-for-graduate-and-professional-students page taken! 

163 of 172
Visiting www.vanderbilt.edu/coronavirus//tagnew
...Taking a screenshot
Screenshot of tagnew page taken! 

164 of 172
Visiting www.vanderbilt.edu/coronavirus//20200611june-11-2020-details-of-research-ramp-up-phase-2-process
...Taking a screenshot
Screenshot of 20200611june-11-2020-details-of-research-ramp-up-phase-2-process page taken! 

165 of 172
Visiting www.vanderbilt.edu/coronavirus//20200616june-16-2020-our-plan-for-the-fall-semester
...Taking a screenshot
Screenshot of 20200616june-16-2020-our-plan-for-the-fall-semester page taken! 

166 of 172
Visiting www.vanderbilt.edu/coronavirus//fall-2020fall-2020-postdocs
...Taking a screenshot
Screenshot of fall-2020fall-2020-postdocs page taken! 

167 of